In [39]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [19]:
dfshp = gpd.read_file('/home/kongo/Desktop/Practica Datos Final/SHP Manzanas Censales/Microdatos_Censo_2017%3A_Manzana.shp',index_col=0,encoding='latin-1',error_bad_lines=False,sep=';')

In [20]:
dfcsv = pd.read_csv('/home/kongo/Desktop/Practica MAIN/CENSO 2017 INE/BASE DE DATOS CENSO 2017/Censo2017_ManzanaEntidad_CSV/Censo2017_Manzanas.csv',index_col=0,encoding='latin-1',error_bad_lines=False,sep=';')

/home/kongo/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [32]:
dfcsv = dfcsv.rename(columns={"ID_MANZENT":"MANZENT"})
df = dfshp.merge(dfcsv,how='left',axis =1,on='MANZENT')

In [35]:
df = dfshp.merge(dfcsv,how='left', on='MANZENT')

In [38]:
#Indice De vejez, (Edad65ymas)/(Total de personas) por manzana censal.

df.EDAD_65YMAS = pd.to_numeric(df.EDAD_65YMAS, errors='coerce').fillna(0)
df.PERSONAS = pd.to_numeric(df.PERSONAS, errors='coerce').fillna(0)

df['IndiceVejez'] = (df.EDAD_65YMAS)/(df.PERSONAS)
df.IndiceVejez = pd.to_numeric(df.IndiceVejez, errors='coerce').fillna(0)

In [40]:
# Razón de envejecimiento, número de personas de 65 años o más respecto al número de personas de menos de 15 años, 

df.EDAD_0A5 = pd.to_numeric(df.EDAD_0A5, errors='coerce').fillna(0)
df.EDAD_6A14 = pd.to_numeric(df.EDAD_6A14, errors='coerce').fillna(0)

df['RazonEnvej'] = (df.EDAD_65YMAS)/(df.EDAD_0A5 + df.EDAD_6A14)
df.RazonEnvej.replace([np.inf, -np.inf], np.nan, inplace=True)
df.RazonEnvej = pd.to_numeric(df.RazonEnvej, errors='coerce').fillna(0)

In [41]:
#índice de dependencia demográfico ,cociente entre la suma de los grupos de población de menos de 15 y de 65 y más años de edad y la población de 15 a 64 años de edad

df.EDAD_15A64 = pd.to_numeric(df.EDAD_15A64, errors='coerce').fillna(0)

df['IndiceDepDem'] = (df.EDAD_0A5 + df.EDAD_6A14)/(df.EDAD_15A64)
df.IndiceDepDem.replace([np.inf, -np.inf], np.nan, inplace=True)
df.IndiceDepDem = pd.to_numeric(df.IndiceDepDem, errors='coerce').fillna(0)

In [42]:
# - la razón de sexos que es cuántos hombres por cuántas mujeres hay en un territorio determinado.

df.HOMBRES = pd.to_numeric(df.HOMBRES, errors='coerce').fillna(0)
df.MUJERES = pd.to_numeric(df.MUJERES, errors='coerce').fillna(0)

df['razonSexos'] = (df.HOMBRES)/(df.MUJERES)
df.razonSexos.replace([np.inf, -np.inf], np.nan, inplace=True)
df.razonSexos = pd.to_numeric(df.razonSexos, errors='coerce').fillna(0)

In [43]:
df.to_file("DatosManzana.shp")

/tmp/ipykernel_110539/2760853496.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  df.to_file("DatosManzana.shp")


In [ ]:
#y algún indicador de posición socioeconómica. 

#indice hacinamiento num casas / personas
#3 sub-indicadores de paredes, suelo y techo. promedio lineal de las variables consideradas insuficientes
#MATIRREC
#Promedio
# P03C_5 piso de tierra
# P03C_4 Materialidad en el piso de capa de cemento sobre tierra
# P03B_6 Materialidad en la cubierta del techo de materiales precarios (lata, cartón, plástico, etc.)
# P03B_7 Materialidad en la cubierta del techo sin cubierta sólida del techo
# P03B_4 techo fonolita
# P015 = Mediagua, mejora, rancho o choza
# P013 = vivienda tradicional indigena
df['IndiceVivienda'] = df[['P03C_5', 'P03B_6','P03C_4','P03B_7']].mean(axis=1)
#Luego, el indicador se normaliza con su inverso aditivo, para asegurar que el valor
#máximo sea lo más deseable y el 0 lo menos deseable
dfViv = (df.IndiceVivienda-df.IndiceVivienda.min())/(df.IndiceVivienda.max()-df.IndiceVivienda.min())
df['IndiceVivienda'] = dfViv


#MATIRREC total de viviendas con materialidad irrecuperable

dfViv = (df.IndiceVivienda-df.IndiceVivienda.min())/(df.IndiceVivienda.max()-df.IndiceVivienda.min())
df['IndiceVivienda'] = dfViv